<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 15.6 MB/s 
     |████████████████████████████████| 182 kB 70.8 MB/s 
     |████████████████████████████████| 168 kB 69.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 76.8 MB/s 
     |████████████████████████████████| 166 kB 51.5 MB/s 
     |████████████████████████████████| 166 kB 69.6 MB/s 
     |████████████████████████████████| 162 kB 56.3 MB/s 
     |████████████████████████████████| 162 kB 58.5 MB/s 
     |████████████████████████████████| 158 kB 78.3 MB/s 
     |████████████████████████████████| 157 kB 79.0 MB/s 
     |████████████████████████████████| 157 kB 79.4 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 74.2 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 74.2 MB/s 
     |████████████████████████████████| 157 kB 73.5 MB/s 
     |██████████

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [4]:
efficientnet = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
    )

model_efficientnet=Sequential([
  # layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  efficientnet,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(5, activation='softmax')
])
efficientnet.trainable=False
model_efficientnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 4,055,976
Trainable params: 6,405
Non-trainable params: 4,049,571
_________________________________________________________________


In [5]:
model_efficientnet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "EfficientnetB0"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_efficientnet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback(compute_flops = True,
    save_model = True)]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.0867 - accuracy: 0.5724

wandb: Adding directory to artifact (/content/wandb/run-20221213_114537-1vyfdqqc/files/model-best)... Done. 0.1s


22/22 [==============================] - 337s 11s/step - loss: 1.0867 - accuracy: 0.5724 - val_loss: 0.6329 - val_accuracy: 0.7943
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.4817 - accuracy: 0.8580

wandb: Adding directory to artifact (/content/wandb/run-20221213_114537-1vyfdqqc/files/model-best)... Done. 0.1s


...

22/22 [==============================] - 36s 2s/step - loss: 0.0258 - accuracy: 0.9986 - val_loss: 0.0641 - val_accuracy: 0.9886
Epoch 37/40
22/22 [==============================] - ETA: 0s - loss: 0.0229 - accuracy: 0.9986

wandb: Adding directory to artifact (/content/wandb/run-20221213_114537-1vyfdqqc/files/model-best)... Done. 0.1s


22/22 [==============================] - 39s 2s/step - loss: 0.0229 - accuracy: 0.9986 - val_loss: 0.0607 - val_accuracy: 0.9943
Epoch 38/40
22/22 [==============================] - ETA: 0s - loss: 0.0221 - accuracy: 0.9972

wandb: Adding directory to artifact (/content/wandb/run-20221213_114537-1vyfdqqc/files/model-best)... Done. 0.1s


22/22 [==============================] - 39s 2s/step - loss: 0.0221 - accuracy: 0.9972 - val_loss: 0.0596 - val_accuracy: 0.9943
Epoch 39/40
22/22 [==============================] - 3s 119ms/step - loss: 0.0212 - accuracy: 0.9972 - val_loss: 0.0599 - val_accuracy: 0.9943
Epoch 40/40
22/22 [==============================] - 3s 117ms/step - loss: 0.0279 - accuracy: 0.9957 - val_loss: 0.0597 - val_accuracy: 0.9943
CPU times: user 21min 45s, sys: 40 s, total: 22min 25s
Wall time: 29min 16s


In [10]:
wandb.alert(title = "Run finished",
            text = "efficientnetb0")

In [11]:
wandb.finish()

accuracy,▁▆▇▇▇███████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇▇▇▇█▇██████████████████████████████
val_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.00064
accuracy,0.99574
best_epoch,37
best_val_loss,0.05957
epoch,39
loss,0.02788


In [12]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
